In [1]:
import os, time, glob, re
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
from datetime import datetime
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import FormRecognizerClient
from azure.ai.formrecognizer import FormTrainingClient
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions


In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [3]:
AZURE_FORM_RECOGNIZER_ENDPOINT = os.getenv('AZURE_FORM_RECOGNIZER_ENDPOINT')
AZURE_FORM_RECOGNIZER_KEY = os.getenv('AZURE_FORM_RECOGNIZER_KEY')

AZURE_STORAGE_CONNECTION_STRING = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

In [4]:
blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)

In [5]:
container_client = blob_service_client.get_container_client('apbk-project')

In [6]:
blob_list = []
for blob_i in container_client.list_blobs():
    blob_list.append(blob_i.name)

In [7]:
blob_list

['FlightManifest.csv']

In [8]:
account_name = os.getenv('account_name')
account_key = os.getenv('account_key')
container_name = 'apbk-project'

In [9]:
for blob_i in blob_list:
    #generate a shared access signature for each blob file
    sas_i = generate_blob_sas(account_name = account_name,
                                container_name = container_name,
                                blob_name = blob_i,
                                account_key=account_key,
                                permission=BlobSasPermissions(read=True),
                                expiry=datetime.utcnow() + timedelta(hours=24))
    
    sas_url = 'https://' + account_name+'.blob.core.windows.net/' + container_name + '/' + blob_i + '?' + sas_i
    
    df = pd.read_csv(sas_url)

In [10]:
df

,Carrier,Flight No,Origin,Destination,Date,Time,First Name,Last Name,Sex,Class,SeatNo,DateofBirth,Baggage,DoBValidation,PersonValidation,LuggageValidation,NameValidation,BoardingPassValidation
0,UA,234,San Francisco,Chicago,20 April 2022,10:00,Avkash,Chauhan,M,E,20A,"January 1, 1990",0,False,False,False,False,False
1,UA,234,San Francisco,Chicago,20 April 2022,10:00,James,Jackson,M,E,25B,"October 12, 1956",1,False,False,False,False,False
2,UA,234,San Francisco,Chicago,20 April 2022,10:00,Libby,Herold,F,B,3D,"February 10, 1996",1,False,False,False,False,False
3,UA,234,San Francisco,Chicago,20 April 2022,10:00,Radha,S Kumar,F,E,34B,"March 5, 1994",1,False,False,False,False,False
4,UA,234,San Francisco,Chicago,20 April 2022,10:00,Sameer,Kumar,M,E,34A,"January 25, 1990",1,False,False,False,False,False
5,UA,234,San Francisco,Chicago,20 April 2022,10:00,James,Webb,M,B,1A,"December 15, 1970",1,False,False,False,False,False


In [11]:
flight_manifest_df = df.drop(df[df['First Name'] == 'Sameer'].index)
flight_manifest_df

,Carrier,Flight No,Origin,Destination,Date,Time,First Name,Last Name,Sex,Class,SeatNo,DateofBirth,Baggage,DoBValidation,PersonValidation,LuggageValidation,NameValidation,BoardingPassValidation
0,UA,234,San Francisco,Chicago,20 April 2022,10:00,Avkash,Chauhan,M,E,20A,"January 1, 1990",0,False,False,False,False,False
1,UA,234,San Francisco,Chicago,20 April 2022,10:00,James,Jackson,M,E,25B,"October 12, 1956",1,False,False,False,False,False
2,UA,234,San Francisco,Chicago,20 April 2022,10:00,Libby,Herold,F,B,3D,"February 10, 1996",1,False,False,False,False,False
3,UA,234,San Francisco,Chicago,20 April 2022,10:00,Radha,S Kumar,F,E,34B,"March 5, 1994",1,False,False,False,False,False
5,UA,234,San Francisco,Chicago,20 April 2022,10:00,James,Webb,M,B,1A,"December 15, 1970",1,False,False,False,False,False


In [12]:
per = flight_manifest_df.loc[(flight_manifest_df['First Name'] == 'Avkash') & (flight_manifest_df['Last Name'] == 'Chauhan')]
per['Carrier'][0]

'UA'

In [13]:
form_recognizer_client = FormRecognizerClient(endpoint=AZURE_FORM_RECOGNIZER_ENDPOINT, credential=AzureKeyCredential(AZURE_FORM_RECOGNIZER_KEY))

In [14]:
form_training_client = FormTrainingClient(endpoint=AZURE_FORM_RECOGNIZER_ENDPOINT, credential=AzureKeyCredential(AZURE_FORM_RECOGNIZER_KEY))
custom_model_info = form_training_client.get_custom_model(model_id='8895791a-f512-49de-bd53-29630e25e2e7')

In [15]:
print("Model ID: {}".format(custom_model_info.model_id))
print("Status: {}".format(custom_model_info.status))
print("Training started on: {}".format(custom_model_info.training_started_on))
print("Training completed on: {}".format(custom_model_info.training_completed_on))

Model ID: 8895791a-f512-49de-bd53-29630e25e2e7
Status: ready
Training started on: 2023-09-09 23:59:52+00:00
Training completed on: 2023-09-09 23:59:53+00:00


In [16]:
digital_ids_path = '../material_preparation_step/digital_id'
boarding_passes_path = '../material_preparation_step/boarding_pass'

In [17]:
# list to store files
res = []

# Iterate directory
for file_path in os.listdir(boarding_passes_path):
    # check if current file_path is a file
    if os.path.isfile(os.path.join(boarding_passes_path, file_path)):
        # add filename to list
        res.append(file_path)
print(res)

['boarding-radha_s-kumar.pdf', 'boarding-james_webb.pdf', 'boarding-libby_herold.pdf', 'boarding-james_jackson.pdf', 'boarding-avkash_chauhan.pdf']


In [18]:
def validate_flight_info_and_identity(boarding_pass_file, digital_id_file, form_recognizer, facial_recognizer):
    with open(boarding_pass_file, "rb") as boarding_pass:
        custom_test_action = form_recognizer.begin_recognize_custom_forms(model_id='8895791a-f512-49de-bd53-29630e25e2e7', form=boarding_pass)
        boarding_pass_result = custom_test_action.result()
    for recognized_content in boarding_pass_result:
        for name, field in recognized_content.fields.items():
            if name == 'Passenger Name':
                passenger_name_from_boarding_pass = field.value
            elif name == 'First Name':
                first_name_from_boarding_pass = field.value
            elif name == 'Last Name':
                last_name_from_boarding_pass = field.value
            elif name == 'Carrier':
                carrier_from_boarding_pass = field.value
            elif name == 'Flight No':
                flight_no_from_boarding_pass = int(field.value)
            elif name == 'Class':
                class_from_boarding_pass = field.value
            elif name == 'From':
                from_from_boarding_pass = field.value
            elif name == 'To':
                to_from_boarding_pass = field.value
            elif name == 'Seat':
                seat_from_boarding_pass = field.value
            elif name == 'Date':
                fligh_date_from_boarding_pass = datetime.strptime(field.value, "%B %d, %Y").date()
            elif name == 'Boarding Time':
                fligh_time_from_boarding_pass = field.value
    
    with open(digital_id_file, "rb") as digital_id:
        poller = form_recognizer.begin_recognize_identity_documents(identity_document=digital_id)
        digital_id_content = poller.result()
    
    person_id_dict = {
        'FirstName': digital_id_content[0].fields.get("FirstName").value,
        'LastName': digital_id_content[0].fields.get("LastName").value,
        'DateOfBirth': digital_id_content[0].fields.get("DateOfBirth").value
    }
    
    if ((first_name_from_boarding_pass).lower() == (person_id_dict['FirstName']).lower() and (last_name_from_boarding_pass).lower() == (person_id_dict['LastName']).lower()):
        flight_manifest_df.loc[(flight_manifest_df['First Name'] == first_name_from_boarding_pass) & (flight_manifest_df['Last Name'] == last_name_from_boarding_pass), 'NameValidation'] = True
        per = flight_manifest_df.loc[(flight_manifest_df['First Name'] == 'Avkash') & (flight_manifest_df['Last Name'] == 'Chauhan')]
    
        if (person_id_dict['DateOfBirth'] == datetime.strptime(per['DateofBirth'][0], "%B %d, %Y").date()):
            flight_manifest_df.loc[(flight_manifest_df['First Name'] == first_name_from_boarding_pass) & (flight_manifest_df['Last Name'] == last_name_from_boarding_pass), 'DoBValidation'] = True
        else:
            print("Dear Sir/Madam, Some of the information on your ID card does not match the flight manifest data, so you cannot board the plane."
                  +"Please see a customer service representative.")
        
        if (carrier_from_boarding_pass == per['Carrier'][0]
            and flight_no_from_boarding_pass == per['Flight No'][0]
            and from_from_boarding_pass == per['Origin'][0]
            and to_from_boarding_pass == per['Destination'][0]
            and fligh_date_from_boarding_pass == datetime.strptime(per['Date'][0], '%d %B %Y').date()
            and fligh_time_from_boarding_pass == per['Time'][0]
            and seat_from_boarding_pass == per['SeatNo'][0]
            and class_from_boarding_pass == per['Class'][0]):
            flight_manifest_df.loc[(flight_manifest_df['First Name'] == first_name_from_boarding_pass) & (flight_manifest_df['Last Name'] == last_name_from_boarding_pass), 'BoardingPassValidation'] = True
            print("Dear Mr. {} {}, You are welcome to flight # {} leaving at {} from {} to {}.".format(first_name_from_boarding_pass,
                                                                                                       last_name_from_boarding_pass,
                                                                                                       flight_no_from_boarding_pass,
                                                                                                       fligh_time_from_boarding_pass,
                                                                                                       from_from_boarding_pass,
                                                                                                       to_from_boarding_pass,)
                  +"Your seat number is {}, and it is confirmed.".format(seat_from_boarding_pass)
                  + "We did not find a prohibited item (lighter) in your carry-on baggage,"
                  +"thanks for following the procedure. Your identity is verified so please board the plane."
                  )
        else:
            print("Dear Sir/Madam, some of the information in your boarding pass does not match the flight manifest data, so you cannot board the plane."
                  +"Please see a customer service representative.")

In [19]:
for boarding_file in res:
    s = boarding_file[9:]
    s = s[:-4]
    ss = s.split("_")
    fn = ss[0]
    ln = ss[1]
    regex = r'[\^ca-dl-]+{fname}+[-]+{lname}'.format(fname=fn, lname=ln)
    for file in os.listdir(digital_ids_path):
        if re.search(regex, file):
            regex = r'{name}'.format(name=ln)
            if re.search(regex, file):
                    print(boarding_file, file)
                    id_file = file
                    if(id_file == "ca-dl-james-webb.png"):
                        continue
                    else:
                        validate_flight_info_and_identity(os.path.join(boarding_passes_path, boarding_file),
                                                      os.path.join(digital_ids_path, id_file), form_recognizer_client, 'facial_recognizer')
            


boarding-radha_s-kumar.pdf ca-dl-radha-s-kumar.png
boarding-james_webb.pdf ca-dl-james-webb.png
boarding-libby_herold.pdf ca-dl-libby-herold.png
Dear Sir/Madam, Some of the information on your ID card does not match the flight manifest data, so you cannot board the plane.Please see a customer service representative.
Dear Sir/Madam, some of the information in your boarding pass does not match the flight manifest data, so you cannot board the plane.Please see a customer service representative.
boarding-james_jackson.pdf ca-dl-james-jackson.png
Dear Sir/Madam, Some of the information on your ID card does not match the flight manifest data, so you cannot board the plane.Please see a customer service representative.
Dear Sir/Madam, some of the information in your boarding pass does not match the flight manifest data, so you cannot board the plane.Please see a customer service representative.
boarding-avkash_chauhan.pdf ca-dl-avkash-chauhan.png
Dear Sir/Madam, some of the information in your

In [20]:
test_pass = os.path.join(boarding_passes_path, 'boarding-avkash_chauhan.pdf')
test_id = os.path.join(digital_ids_path, 'ca-dl-avkash-chauhan.png')
validate_flight_info_and_identity(test_pass, test_id, form_recognizer_client, 'facial_recogn')

Dear Sir/Madam, some of the information in your boarding pass does not match the flight manifest data, so you cannot board the plane.Please see a customer service representative.


In [21]:
flight_manifest_df

,Carrier,Flight No,Origin,Destination,Date,Time,First Name,Last Name,Sex,Class,SeatNo,DateofBirth,Baggage,DoBValidation,PersonValidation,LuggageValidation,NameValidation,BoardingPassValidation
0,UA,234,San Francisco,Chicago,20 April 2022,10:00,Avkash,Chauhan,M,E,20A,"January 1, 1990",0,True,False,False,True,False
1,UA,234,San Francisco,Chicago,20 April 2022,10:00,James,Jackson,M,E,25B,"October 12, 1956",1,False,False,False,True,False
2,UA,234,San Francisco,Chicago,20 April 2022,10:00,Libby,Herold,F,B,3D,"February 10, 1996",1,False,False,False,True,False
3,UA,234,San Francisco,Chicago,20 April 2022,10:00,Radha,S Kumar,F,E,34B,"March 5, 1994",1,False,False,False,False,False
5,UA,234,San Francisco,Chicago,20 April 2022,10:00,James,Webb,M,B,1A,"December 15, 1970",1,False,False,False,False,False
